This script gathers perpetual, spot and funding rate data from coingecko API

In [ ]:
import requests
import pandas as pd
import datetime

perp = f'https://fapi.binance.com/fapi/v1/continuousKlines?pair={symbol}&contractType={contract}&interval={interval}&endTime={endtime}'
spot = f'https://api.binance.com/api/v3/klines?symbol={symbol}&interval={interval}&endTime={endtime}'
index = f'https://fapi.binance.com/fapi/v1/indexPriceKlines?pair={symbol}&contractType={contract}&interval={interval}'
mark = f'https://fapi.binance.com/fapi/v1/markPriceKlines?symbol={symbol}&interval={interval}'
rate = f'https://fapi.binance.com/fapi/v1/fundingRate?symbol={symbol}&endTime={endtime}&limit=500'
#interest = f'https://api.binance.com/sapi/v1/lending/union/interestHistory?lendingType=DAILY&asset=BTC'


In [ ]:
def process_trading(raw):
#trading data
    for i in range(len(raw)):
                    epoch = int(raw[i][0]/1000)
                    time = datetime.datetime.utcfromtimestamp(int(raw[i][0]/1000)) #converting epoch from ms to seconds

                    opn = raw[i][1]
                    hgh = raw[i][2]
                    low = raw[i][3]
                    cls = raw[i][4]
                    volume = raw[i][5]
                    usd_volume = raw[i][7]
                    trades = raw[i][8]

                    lst.append([epoch, time, opn, hgh, low, cls, volume, usd_volume, trades])
    return

## spot

In [ ]:
lst = []
endtime = int(datetime.datetime.now().strftime('%s')) * 1000

In [ ]:
cont = True

while cont:
    query = f'https://api.binance.com/api/v3/klines?symbol={symbol}&interval={interval}&endTime={endtime}'
    response = requests.get(query)
    raw = response.json()
    process_trading(raw)
    endtime = raw[0][0]
    if len(raw) < 500:
        cont = False

In [ ]:
df_trading = pd.DataFrame(lst, columns=['epoch', 'timestamp', 'open', 'high', 'low', 'close', 'volume', 'usd_volume', 'trades'])
df = df_trading.copy()
df.sort_values(by=['epoch'], ascending=True, inplace=True)
df = df.drop_duplicates(subset=['epoch'], keep='first')
df = df.reset_index(drop=True)
df

In [ ]:
#store in csv
df.to_csv(f'spot_{currencies}_usdt_1h.csv', index=False)

print('spot_', currencies, '.csv created')
print(currencies, 'first row', df.iloc[0])
print(currencies, 'last row', df.iloc[-1])
print('')

## futures api

In [ ]:
lst = []
endtime = int(datetime.datetime.now().strftime('%s')) * 1000

In [ ]:
cont = True

while cont:
    query = f'https://fapi.binance.com/fapi/v1/continuousKlines?pair={symbol}&contractType={contract}&interval={interval}&endTime={endtime}'
    response = requests.get(query)
    raw = response.json()
    process_trading(raw)
    endtime = raw[0][0]
    if len(raw) < 500:
        cont = False

In [ ]:
df_trading = pd.DataFrame(lst, columns=['epoch', 'timestamp', 'open', 'high', 'low', 'close', 'volume', 'usd_volume', 'trades'])
df = df_trading.copy()
df.sort_values(by=['epoch'], ascending=True, inplace=True)
df = df.drop_duplicates(subset=['epoch'], keep='first')
df = df.reset_index(drop=True)

In [ ]:
#store in csv
df.to_csv(f'perp_{currencies}_usdt_1h.csv', index=False)

print('perp_', currencies, '.csv created')
print(currencies, 'first row', df.iloc[0])
print(currencies, 'last row', df.iloc[-1])
print('')

## funding rate

In [ ]:
def process_funding(raw):
#funding rate
    for i in range(len(raw)):
                    epoch = int(raw[i]['fundingTime']/1000)
                    time = datetime.datetime.utcfromtimestamp(int(raw[i]['fundingTime']/1000)) #converting epoch from ms to seconds
                    rate = raw[i]['fundingRate']

                    lst.append([epoch, time, rate])
    return

In [ ]:
lst = []
starttime = 1559347200000 #'2019-06-01 00:00:00'

In [ ]:
cont = True

while cont:
    query = f'https://fapi.binance.com/fapi/v1/fundingRate?symbol={symbol}&startTime={starttime}&limit=500'
    response = requests.get(query)
    raw = response.json()
    process_funding(raw)
    starttime = raw[-1]['fundingTime']
    if len(raw) < 500: 
        cont = False

In [ ]:
df_funding = pd.DataFrame(lst, columns=['epoch', 'timestamp', 'funding_rate'])
df = df_funding.copy()
df.sort_values(by=['epoch'], ascending=True, inplace=True)
df = df.drop_duplicates(subset=['epoch'], keep='first')
df = df.reset_index(drop=True)

In [ ]:
#store in csv
df.to_csv(f'rate_{currencies}_usdt.csv', index=False)

print('funding_rate_', currencies, '.csv created')
print(currencies, 'first row', df.iloc[0])
print(currencies, 'last row', df.iloc[-1])
print('')